In [6]:
import numpy as np
import cv2
import random
#from random import choice
#from random import randint
#from random import uniform

from skimage.transform import rotate
from skimage.transform import warp
from skimage.transform import ProjectiveTransform



def pespective(input, mascara):
    #cv2.imshow('entrada', input)
    cols = input.shape[1]
    rows = input.shape[0]
    inputQuad = np.array([[0, 0],[cols, 0],[cols,rows],[0,rows]], np.float32)
    outputQuad = np.array([[-randint(0,17),-randint(0,17)],[cols+randint(0,17),-randint(0,17)],	
	[cols+randint(0,17), rows+randint(0,17)],[-randint(0,17),rows+randint(0,17)]], np.float32)
    M = cv2.getPerspectiveTransform(outputQuad, inputQuad)
    output = cv2.warpPerspective(input,M,(cols,rows)) 
    output2 = cv2.warpPerspective(mascara,M,(cols,rows)) 
    #cv2.imshow("Input",output)
    #cv2.waitKey(0);
    return [output,output2]

'''
Aplica rotacao 2D e transformacao de pespectiva nas placas nas placas
Tambem muda a intensidade de alguns pixels da placa com a finalidade 
de deixar a regiao da coroa circular mais escura, assim como o interior  
mais claro da placa.
'''

def transformation(name, name2):

    # read in grayscale
    src1 = cv2.imread(name, 0)
    mask = cv2.imread(name2, 0)
    '''
    Aplicado para criar certo ruido (noise) na imagem
    '''
    rows, cols = src1.shape
    k = 0
    for i in range(0, rows):
        for j in range(0, cols):
            editValue = src1[i, j]
            var = randint(25, 100)
            k = var
            if editValue > 60 and editValue < 160:
                src1[i, j] = var
            elif editValue > 230 and editValue < 256:
                src1[i, j] = 255 - var
    if k > 40:
        getElement = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        src1 = cv2.erode(src1, getElement)

        getElement = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        src1 = cv2.dilate(src1, getElement)

    '''
    rotation transformation (-20 ate 20 graus)
    '''
    image_center = tuple(np.array(src1.shape) / 2)
    angle = randint(-30, 30)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(src1, rot_mat, src1.shape, flags=cv2.INTER_LINEAR)
    result2 = cv2.warpAffine(mask, rot_mat, mask.shape, flags=cv2.INTER_LINEAR)     


    '''
    perspective transformation
    '''
    img = result
    rows = img.shape[0]
    cols = img.shape[1]
    

    # retangulo da imagem

    # [[0, 0], [160, 40], [0, 200], [160, 160]]

    # [[0, 0], [0, 200], [200, 0], [200, 200]]

    pts1 = np.float32([[0, 0], [0, 200], [200, 0], [200, 200]])

    seed = randint(0, 1)
    variation1 = randint(0, 8)
    variation2 = randint(15, 25)

    # perspectiva para a direita
    if seed == 0:
        pts2 = np.float32([[(0 + variation1), 0], [(0 + variation1),
                                                   200], [(200 - variation2), 0], [(200 - variation2), 200]])
    # perspectiva para a esquerda
    else:
        pts2 = np.float32([[(0 + variation2), 0], [(0 + variation2),
                                                   200], [(200 - variation1), 0], [(200 - variation1), 200]])

    M = cv2.getPerspectiveTransform(pts1, pts2)
    dst = cv2.warpPerspective(img, M, (200, 200))
    dst2 = cv2.warpPerspective(result2, M, (200, 200))
    #cv2.imshow("ois", mask)
    #cv2.waitKey(0)
    return [dst, dst2]





# aplica mudanca de brilho, contraste, bluring e resize nas imagens
def variation(image,mascara, pasta, arquivo, transf):
    #image = cv2.cvtColor(image,  cv2.COLOR_BGR2GRAY)
    # escolher valores aleatorios para alpha (simple contrast control) (0.4 -
    # 0.9)
    alpha = uniform(.4, .9)

    # escolher valores aleatorios para beta (simple brightness control) (10 -
    # 20)
    beta = randint(12, 16)
    cv2.convertScaleAbs(image, image, alpha, beta)

    # add gaussian blurring
    size = int(choice('35'))
    blur = cv2.GaussianBlur(image, (size, size), 0, 0)
    #blur = cv2.bilateralFilter(blur,9,75,75)
    uhu =  cv2.medianBlur(blur,5)    
    #uhu2 = cv2.GaussianBlur(uhu, (0, 0), 10)
    #uhu3 = cv2.addWeighted(uhu2,1.5,-0.5, 0,)



    
    th, mask_th = cv2.threshold(mascara, 50, 255, cv2.THRESH_BINARY);
    blur = cv2.bitwise_and(uhu, mask_th)


    #Aplicar filtro de mediana:
    

    # resize
    final = cv2.resize(blur, (56, 56))
  
    
    cv2.imwrite("Generated/" + str(pasta)+ "/"+ str(arquivo) + " "+str(transf) + ".png", final)

'''
definicao criada para gerar base de treino
para cada classe, aplicar transformacoes e sobreposicao com background
sao geradas 1300 imagens por classe
'''


def create_database():
    flag_name = 0
    for i in range(0, 1):#30): CRIAR IDENT DEPOIS DISSO
        filepath = './traffic_signs_classes/' + str(i) + '.png'
        filepath2 = './traffic_signs_classes/thresholding/' + str(i) + '.jpg'
        for j in range(0, 1):# 25):
            src, mask = transformation(filepath, filepath2)
            for transf in range(0, 1):#20):
                src2, mask2 = pespective(src, mask)
                variation(src2, mask, i, j, transf)
                print('classe: ' + str(i) + ' image : ' + str(j) + ' transf : ' + str(transf))


'''
definicao criada para gerar base de teste
para cada classe, aplicar transformacoes e sobreposicao com background
sao geradas 520 imagens por classe = 40% * base de treino
'''

create_database()


classe: 0 image : 0 transf : 0
